In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import sqlite3
import os

In [2]:
def generate_id(prefix, num):
    return f"{prefix}{num:03d}"

def random_string(length):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return ''.join(random.choice(letters) for i in range(length))

def query_db(query, params=()):
    conn = sqlite3.connect('clash_royale.db')
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    return df

In [3]:
escala = 10
# Para cada grau desta escala são criados
# 4 players, 10 battles, 20 battleplayers, 20 decks, 160 deckcards
# Refiro-me, em cada um dos casos acima, a entidades distintas.
# O número de cartas distintas criadas é sempre 20

# Generate players
players = [
    {
        "playerTag": generate_id("PL", i),
        "name": f"Player{i}",
        "expLevel": random.randint(1, 13),
        "trophies": random.randint(0, 6000),
        "bestTrophies": random.randint(0, 6000),
        "wins": random.randint(0, 5000),
        "losses": random.randint(0, 5000),
        "battleCount": random.randint(0, 10000),
        "threeCrownWins": random.randint(0, 1000),
        "challengeCardsWon": random.randint(0, 1000),
        "challengeMaxWins": random.randint(0, 12),
        "tournamentCardsWon": random.randint(0, 1000),
        "tournamentBattleCount": random.randint(0, 1000),
        "role": random.choice(["member", "elder", "co-leader", "leader"]),
        "donations": random.randint(0, 5000),
        "donationsReceived": random.randint(0, 5000),
        "totalDonations": random.randint(0, 10000),
        "warDayWins": random.randint(0, 100),
        "clanCardsCollected": random.randint(0, 10000)
    }
    for i in range(1, escala * 4 + 1)
]

# Generate cards
cards = [
    {
        "cardId": generate_id("C", i),
        "name": f"Card{i}",
        "maxLevel": random.randint(1, 13),
        "iconUrl": f"http://example.com/card{i}.png",
        "rarity": random.choice(["common", "rare", "epic", "legendary"])
    }
    for i in range(1, 21) # escala * 20 + 1
]

# Generate decks
decks = [
    {
        "deckId": generate_id("D", i)
    }
    for i in range(1, escala * 20 + 1)
]

# Generate deck cards ensuring each deck has 8 unique cards
deck_cards = []
for deck in decks:
#     card_ids = random.sample([card["cardId"] for card in cards], 8)
    card_ids = list(np.random.choice([card["cardId"] for card in cards], 8, replace=True))
    for card_id in card_ids:
        deck_cards.append({
            "deckCardId": generate_id("DC", len(deck_cards) + 1),
            "deckId": deck["deckId"],
            "cardId": card_id,
            "cardLevel": random.randint(1, 13)
        })

# Generate battles
# Não escolhe ainda o vencedor.
battles = [
    {
        "battleId": generate_id("B", i),
        "battleTime": datetime.now() - timedelta(days=random.randint(0, 365)),
        "gameMode": random.choice(["1v1", "2v2"]),
        "deckSelection": random.choice(["predefined", "custom"]),
#         "winner": random.choice([f"PL{j:03d}" for j in range(1, escala * 4 + 1)]),
        "trophyChange": random.randint(-30, 30),
        "crowns": random.randint(0, 3),
        "arena": random.choice(["Training Camp", "Goblin Stadium", "Bone Pit", "Barbarian Bowl"])
    }
    for i in range(1, escala * 10 + 1)
]

# Generate battle players ensuring each battle has 2 players
battle_players = []
for battle in battles:
    for _ in range(2):
        player = random.choice(players)
        deck = random.choice(decks)
        battle_players.append({
            "battlePlayerId": generate_id("BP", len(battle_players) + 1),
            "battleId": battle["battleId"],
            "playerTag": player["playerTag"],
            "startingTrophies": random.randint(0, 6000),
            "crownsEarned": random.randint(0, 3),
            "kingTowerHitPoints": random.randint(0, 5000),
            "princessTowerHitPoints": f"{random.randint(0, 5000)},{random.randint(0, 5000)}",
            "clanName": f"Clan{random.randint(1, 10)}",
            "clanTag": random_string(5),
            "deckId": deck["deckId"]
        })
    battle['winner'] = random.choice([battle_players[-1]['playerTag'], battle_players[-2]['playerTag']])

# Apenas para facilitar visualização dos dados sintéticos criados:
df_players = pd.DataFrame(players)
df_cards = pd.DataFrame(cards)
df_decks = pd.DataFrame(decks)
df_deck_cards = pd.DataFrame(deck_cards)
df_battles = pd.DataFrame(battles)
df_battle_players = pd.DataFrame(battle_players)

In [4]:
# Esta célula cria o arquivo clash_royale.db com os dados sintéticos criados

# Primeiramente, apaga o arquivo clash_royale.db, caso existente, para criar um novo
if os.path.exists('clash_royale.db'):
    # Apaga o arquivo
    os.remove('clash_royale.db')

# Cria a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Cria as tabelas
cursor.execute('''
CREATE TABLE IF NOT EXISTS players (
    playerTag TEXT PRIMARY KEY,
    name TEXT,
    expLevel INTEGER,
    trophies INTEGER,
    bestTrophies INTEGER,
    wins INTEGER,
    losses INTEGER,
    battleCount INTEGER,
    threeCrownWins INTEGER,
    challengeCardsWon INTEGER,
    challengeMaxWins INTEGER,
    tournamentCardsWon INTEGER,
    tournamentBattleCount INTEGER,
    role TEXT,
    donations INTEGER,
    donationsReceived INTEGER,
    totalDonations INTEGER,
    warDayWins INTEGER,
    clanCardsCollected INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS cards (
    cardId TEXT PRIMARY KEY,
    name TEXT,
    maxLevel INTEGER,
    iconUrl TEXT,
    rarity TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS decks (
    deckId TEXT PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS deck_cards (
    deckCardId TEXT PRIMARY KEY,
    deckId TEXT,
    cardId TEXT,
    cardLevel INTEGER,
    FOREIGN KEY(deckId) REFERENCES decks(deckId),
    FOREIGN KEY(cardId) REFERENCES cards(cardId)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS battles (
    battleId TEXT PRIMARY KEY,
    battleTime TEXT,
    gameMode TEXT,
    deckSelection TEXT,
    winner TEXT,
    trophyChange INTEGER,
    crowns INTEGER,
    arena TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS battle_players (
    battlePlayerId TEXT PRIMARY KEY,
    battleId TEXT,
    playerTag TEXT,
    startingTrophies INTEGER,
    crownsEarned INTEGER,
    kingTowerHitPoints INTEGER,
    princessTowerHitPoints TEXT,
    clanName TEXT,
    clanTag TEXT,
    deckId TEXT,
    FOREIGN KEY(battleId) REFERENCES battles(battleId),
    FOREIGN KEY(playerTag) REFERENCES players(playerTag),
    FOREIGN KEY(deckId) REFERENCES decks(deckId)
)
''')

# Insere os dados nas tabelas
for player in players:
    cursor.execute('''
    INSERT INTO players VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(player.values()))

for card in cards:
    cursor.execute('''
    INSERT INTO cards VALUES (?, ?, ?, ?, ?)
    ''', tuple(card.values()))

for deck in decks:
    cursor.execute('''
    INSERT INTO decks VALUES (?)
    ''', (deck['deckId'],))

for deck_card in deck_cards:
    cursor.execute('''
    INSERT INTO deck_cards VALUES (?, ?, ?, ?)
    ''', tuple(deck_card.values()))

for battle in battles:
    cursor.execute('''
    INSERT INTO battles VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        battle['battleId'],
        battle['battleTime'].isoformat(),
        battle['gameMode'],
        battle['deckSelection'],
        battle['winner'],
        battle['trophyChange'],
        battle['crowns'],
        battle['arena']
    ))

for battle_player in battle_players:
    cursor.execute('''
    INSERT INTO battle_players VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(battle_player.values()))

# Salva (commit) as mudanças e fecha a conexão com o banco de dados
conn.commit()
conn.close()


In [5]:
################################
####### TESTE DE QUERY 1 #######
################################

# Reabre a conexão com o banco de dados
conn = sqlite3.connect('clash_royale.db')
cursor = conn.cursor()

# Executa a consulta SQL
query = '''
SELECT *
FROM battle_players bp
INNER JOIN decks d ON bp.deckId = d.deckId;
'''
cursor.execute(query)

# Obtém os resultados da consulta
results = cursor.fetchall()

# Imprime os resultados
for row in results:
    print(row)

# Fecha a conexão com o banco de dados
conn.close()


('BP001', 'B001', 'PL010', 3695, 1, 4630, '2750,3127', 'Clan9', 'WYOXY', 'D027', 'D027')
('BP002', 'B001', 'PL023', 4534, 0, 4287, '858,2238', 'Clan4', 'CHGGO', 'D064', 'D064')
('BP003', 'B002', 'PL002', 778, 1, 1836, '2742,2268', 'Clan6', 'EZBBW', 'D152', 'D152')
('BP004', 'B002', 'PL031', 2905, 3, 961, '625,4708', 'Clan10', 'NTRJI', 'D168', 'D168')
('BP005', 'B003', 'PL035', 4793, 1, 4261, '3808,411', 'Clan1', 'SXKSY', 'D062', 'D062')
('BP006', 'B003', 'PL001', 169, 3, 4146, '3891,2877', 'Clan10', 'URILH', 'D012', 'D012')
('BP007', 'B004', 'PL009', 1670, 1, 854, '113,1624', 'Clan7', 'WEYPB', 'D041', 'D041')
('BP008', 'B004', 'PL001', 3182, 1, 4671, '3078,1420', 'Clan1', 'XRNUR', 'D179', 'D179')
('BP009', 'B005', 'PL022', 1637, 2, 389, '4954,4462', 'Clan2', 'VHGSM', 'D092', 'D092')
('BP010', 'B005', 'PL018', 3614, 2, 845, '834,7', 'Clan7', 'DUXJZ', 'D142', 'D142')
('BP011', 'B006', 'PL021', 2946, 2, 917, '4831,4205', 'Clan2', 'AKHVM', 'D169', 'D169')
('BP012', 'B006', 'PL021', 5011, 3

In [6]:
################################
####### TESTE DE QUERY 2 #######
################################

# Função para obter deck_cards associados a um battle_player específico
def get_deck_cards_for_battle_player(battlePlayerId):
    # Reabre a conexão com o banco de dados
    conn = sqlite3.connect('clash_royale.db')
    cursor = conn.cursor()

    # Consulta SQL
    query = '''
    SELECT dc.*
    FROM battle_players bp
    INNER JOIN decks d ON bp.deckId = d.deckId
    INNER JOIN deck_cards dc ON d.deckId = dc.deckId
    WHERE bp.battlePlayerId = ?;
    '''

    # Executa a consulta com o battlePlayerId especificado
    cursor.execute(query, (battlePlayerId,))
    results = cursor.fetchall()

    # Fecha a conexão com o banco de dados
    conn.close()

    return results

# Exemplo de uso da função
battlePlayerId = 'BP001'  # Substitua pelo battlePlayerId desejado
deck_cards = get_deck_cards_for_battle_player(battlePlayerId)

# Imprime os resultados
for deck_card in deck_cards:
    print(deck_card)


('DC209', 'D027', 'C004', 13)
('DC210', 'D027', 'C012', 8)
('DC211', 'D027', 'C014', 11)
('DC212', 'D027', 'C018', 13)
('DC213', 'D027', 'C015', 13)
('DC214', 'D027', 'C005', 8)
('DC215', 'D027', 'C002', 4)
('DC216', 'D027', 'C009', 13)


<h3>Tarefa 1: Calcular a porcentagem de vitórias e derrotas utilizando a carta X em um intervalo de timestamps</h3>

In [36]:
def win_loss_percentage_with_card(card_id, start_time, end_time):
    query = '''
    SELECT bp.playerTag, b.winner
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId = ? AND b.battleTime BETWEEN ? AND ?
    '''
    df = query_db(query, (card_id, start_time, end_time))

    total_battles = len(df)
    if total_battles == 0:
        return {"win_percentage": 0, "loss_percentage": 0}

    wins = df[df['playerTag'] == df['winner']].shape[0]
    losses = total_battles - wins

    win_percentage = (wins / total_battles) * 100
    loss_percentage = (losses / total_battles) * 100

    return {"win_percentage": win_percentage, "loss_percentage": loss_percentage}

# Exemplo de uso:
card_id = 'C001'
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(win_loss_percentage_with_card(card_id, start_time, end_time))

{'win_percentage': 41.75824175824176, 'loss_percentage': 58.24175824175825}


<h3>Tarefa 2: Listar os decks completos que produziram mais de X% de vitórias em um intervalo de timestamps</h3>

In [8]:
def decks_with_high_win_rate(win_rate_threshold, start_time, end_time):
    query = '''
    SELECT bp.deckId, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    WHERE b.battleTime BETWEEN ? AND ?
    GROUP BY bp.deckId
    HAVING (wins * 1.0 / total_battles) * 100 > ?
    '''
    df = query_db(query, (start_time, end_time, win_rate_threshold))
    return df

# Exemplo de uso:
win_rate_threshold = 50  # decks com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(decks_with_high_win_rate(win_rate_threshold, start_time, end_time))


   deckId  total_battles  wins
0    D002              2     2
1    D008              1     1
2    D010              2     2
3    D012              3     2
4    D018              3     2
..    ...            ...   ...
56   D178              1     1
57   D185              1     1
58   D189              1     1
59   D195              1     1
60   D199              2     2

[61 rows x 3 columns]


<h3>Tarefa 3: Calcular a quantidade de derrotas utilizando um combo de cartas em um intervalo de timestamps</h3>

In [34]:
def losses_with_card_combo(card_ids, start_time, end_time):
    card_placeholders = ', '.join('?' for _ in card_ids)
    query = f'''
    SELECT bp.battleId, COUNT(DISTINCT dc.cardId) as card_count
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId IN ({card_placeholders}) AND b.battleTime BETWEEN ? AND ?
    GROUP BY bp.battleId
    HAVING card_count = ?
    '''
    df = query_db(query, (*card_ids, start_time, end_time, len(card_ids)))

    total_battles = len(df)
    losses = total_battles - df[df['battleId'].isin(df['battleId'])].shape[0]

    return {"losses": losses}

# Exemplo de uso:
card_ids = ['C001', 'C002']
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(losses_with_card_combo(card_ids, start_time, end_time))


{'losses': 0}


<h3>Tarefa 4: Calcular a quantidade de vitórias envolvendo a carta X onde o vencedor possui Z% menos troféus que o perdedor, a partida durou menos de 2 minutos, e o perdedor derrubou ao menos duas torres</h3>

In [10]:
def wins_with_card_under_conditions(card_id, trophy_diff_percentage, start_time, end_time):
    query = '''
    SELECT bp.battleId, bp.playerTag, bp.startingTrophies, b.winner, b.battleTime,
           (bp.kingTowerHitPoints + bp.princessTowerHitPoints) as totalHitPoints
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE dc.cardId = ? AND b.battleTime BETWEEN ? AND ? AND totalHitPoints < 2
    '''
    df = query_db(query, (card_id, start_time, end_time))
    
    # Filtrando as condições adicionais
    filtered_df = df[(df['winner'] == df['playerTag']) & 
                     ((df['startingTrophies'] * (1 - trophy_diff_percentage / 100)) > df['startingTrophies']) & 
                     (pd.to_datetime(df['battleTime']) - pd.to_datetime(df['battleTime']) < pd.Timedelta(minutes=2))]
    
    wins = filtered_df.shape[0]
    
    return {"wins": wins}

# Exemplo de uso:
card_id = 'C001'
trophy_diff_percentage = 20
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(wins_with_card_under_conditions(card_id, trophy_diff_percentage, start_time, end_time))


{'wins': 0}


<h3>Tarefa 5: Listar o combo de cartas de tamanho N que produziram mais de Y% de vitórias em um intervalo de timestamps</h3>

In [17]:
def card_combos_with_high_win_rate(combo_size, win_rate_threshold, start_time, end_time):
    query = f'''
    SELECT bp.deckId, dc.cardId, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM battle_players bp
    INNER JOIN battles b ON bp.battleId = b.battleId
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    WHERE b.battleTime BETWEEN ? AND ?
    GROUP BY bp.deckId, dc.cardId
    HAVING COUNT(DISTINCT dc.cardId) = ? AND (wins * 1.0 / total_battles) * 100 > ?
    '''
    df = query_db(query, (start_time, end_time, combo_size, win_rate_threshold))
    return df

# Exemplo de uso:
combo_size = 2
win_rate_threshold = 50  # combos com mais de 50% de vitórias
start_time = '2023-01-01T00:00:00'
end_time = '2024-12-31T23:59:59'
print(card_combos_with_high_win_rate(combo_size, win_rate_threshold, start_time, end_time))


Empty DataFrame
Columns: [deckId, cardId, total_battles, wins]
Index: []


<h3>Consultas adicionais I - Taxa de vitórias por raridade da carta</h3>
<p><i>Nesta consulta analisaremos o desempenho de cartas de diferentes raridades (comum, rara, épica, lendária) em todas as batalhas. A proposta é auxiliar a identificar se cartas de raridades específicas apresentam desempenho superior ou inferior.</i></p>

In [12]:
def win_rate_by_rarity():
    query = '''
    SELECT c.rarity, COUNT(*) as total_battles,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM cards c
    INNER JOIN deck_cards dc ON c.cardId = dc.cardId
    INNER JOIN battle_players bp ON dc.deckId = bp.deckId
    INNER JOIN battles b ON bp.battleId = b.battleId
    GROUP BY c.rarity
    '''
    df = query_db(query)
    
    df['win_rate'] = (df['wins'] / df['total_battles']) * 100
    return df

# Exemplo de uso:
print(win_rate_by_rarity())


      rarity  total_battles  wins   win_rate
0     common            538   276  51.301115
1       epic            425   239  56.235294
2  legendary            478   265  55.439331
3       rare            159    76  47.798742


<h3>Consultas adicionais II - Análise de diversidade de decks</h3>
<p><i>Nesta consulta examinaremos a variedade dos decks utilizados em partidas com alta taxa de troféus. A baixa diversidade de cartas no deck pode ser um indício de que certas combinações se tornam muito poderosas e precisam de equilíbrio.</i></p>

In [13]:
def deck_diversity(high_trophy_threshold):
    query = '''
    SELECT bp.deckId, COUNT(DISTINCT dc.cardId) as unique_cards
    FROM battle_players bp
    INNER JOIN deck_cards dc ON bp.deckId = dc.deckId
    INNER JOIN players p ON bp.playerTag = p.playerTag
    WHERE p.trophies > ?
    GROUP BY bp.deckId
    '''
    df = query_db(query, (high_trophy_threshold,))
    
    return df

# Exemplo de uso:
high_trophy_threshold = 5000  # Ajuste este valor conforme necessário
print(deck_diversity(high_trophy_threshold))


   deckId  unique_cards
0    D002             6
1    D011             7
2    D012             7
3    D019             5
4    D027             8
5    D037             5
6    D041             6
7    D046             8
8    D047             8
9    D064             6
10   D073             8
11   D074             7
12   D079             6
13   D083             7
14   D086             7
15   D089             6
16   D091             7
17   D109             7
18   D110             8
19   D118             5
20   D131             6
21   D142             6
22   D143             8
23   D144             6
24   D145             6
25   D148             8
26   D152             4
27   D154             7
28   D161             6
29   D162             7
30   D163             5
31   D179             7
32   D180             5
33   D190             6
34   D193             8
35   D196             6
36   D197             6
37   D199             6


<h3>Consultas adicionais III - Taxa de uso do card vs taxa de vitórias</h3>
<p><i>Nesta consulta compararemos a frequência de uso de um card com sua taxa de ganho. Cards com altas taxas de utilização mas com baixas taxas de ganho podem ser consideradas mais fortes do que realmente são, enquanto cards com baixas taxas de utilização mas altas taxas de vitórias podem ser subvalorizados. </i></p>

In [14]:
def card_usage_vs_win_rate():
    query = '''
    SELECT c.cardId, c.name, c.rarity,
           COUNT(*) as usage_count,
           SUM(CASE WHEN bp.playerTag = b.winner THEN 1 ELSE 0 END) as wins
    FROM cards c
    INNER JOIN deck_cards dc ON c.cardId = dc.cardId
    INNER JOIN battle_players bp ON dc.deckId = bp.deckId
    INNER JOIN battles b ON bp.battleId = b.battleId
    GROUP BY c.cardId
    '''
    df = query_db(query)
    
    df['win_rate'] = (df['wins'] / df['usage_count']) * 100
    return df

# Exemplo de uso:
print(card_usage_vs_win_rate())


   cardId    name     rarity  usage_count  wins   win_rate
0    C001   Card1  legendary           91    38  41.758242
1    C002   Card2       epic           85    52  61.176471
2    C003   Card3  legendary           77    48  62.337662
3    C004   Card4  legendary           90    44  48.888889
4    C005   Card5     common           85    45  52.941176
5    C006   Card6       epic           63    32  50.793651
6    C007   Card7  legendary           73    46  63.013699
7    C008   Card8     common           94    43  45.744681
8    C009   Card9       epic           99    57  57.575758
9    C010  Card10       rare           87    47  54.022989
10   C011  Card11     common           75    34  45.333333
11   C012  Card12       epic           84    39  46.428571
12   C013  Card13     common           59    33  55.932203
13   C014  Card14     common           70    40  57.142857
14   C015  Card15  legendary           77    42  54.545455
15   C016  Card16       rare           72    29  40.2777